<a href="https://colab.research.google.com/github/gitswathig/assignment3/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#install libraries

In [1]:
!pip install imbalanced-learn


#upload ZIP File

In [2]:
from google.colab import files
uploaded = files.upload()

# After uploading, type the exact file name:
ZIP_FILE = list(uploaded.keys())[0]
print("Uploaded ZIP:", ZIP_FILE)


Saving creditcard.csv.zip to creditcard.csv.zip
Uploaded ZIP: creditcard.csv.zip


extracting ZIP File

In [15]:
import zipfile, os, glob

EXTRACT_DIR = "./dataset"

with zipfile.ZipFile(ZIP_FILE, 'r') as z:
    z.extractall(EXTRACT_DIR)

print(" Extracted ZIP")

# Look for CSVs
csv_files = glob.glob(EXTRACT_DIR + "/**/*.csv", recursive=True)
if not csv_files:
    raise FileNotFoundError("No CSV files found!")

# Prefer creditcard.csv
csv_path = None
for f in csv_files:
    if "creditcard" in f.lower():
        csv_path = f
        break

# Else choose largest CSV
if csv_path is None:
    csv_path = max(csv_files, key=os.path.getsize)

print(" Using CSV:", csv_path)


 Extracted ZIP
 Using CSV: ./dataset/creditcard.csv


load dataset

In [4]:
import pandas as pd

df = pd.read_csv(csv_path)
print("Shape:", df.shape)
df.head()


Shape: (284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


train test split

In [5]:
from sklearn.model_selection import train_test_split
from collections import Counter

TARGET = "Class"
FEATURES = [c for c in df.columns if c != TARGET]

X = df[FEATURES]
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

print("Train class distribution:", Counter(y_train))
print("Test class distribution:", Counter(y_test))


Train class distribution: Counter({0: 213236, 1: 369})
Test class distribution: Counter({0: 71079, 1: 123})


feature scaling

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


handling imbalance SMOTE

In [7]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train_scaled, y_train)

print("After SMOTE:", Counter(y_res))


After SMOTE: Counter({0: 213236, 1: 213236})


Train logistic regression

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_res, y_res)


LogisticRegression(max_iter=1000)

train random forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=40,        # reduced from 150
    max_depth=12,          # limit tree depth
    n_jobs=-1,             # use all CPU cores
    class_weight='balanced'
)

rf.fit(X_res, y_res)



RandomForestClassifier(class_weight='balanced', max_depth=12, n_estimators=40,
                       n_jobs=-1)

evaluating both the models

In [11]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, average_precision_score

def evaluate(model, X_test, y_test, name):
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    print(f"\n===== {name} =====")
    print("ROC-AUC:", roc_auc_score(y_test, y_proba))
    print("PR-AUC:", average_precision_score(y_test, y_proba))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

evaluate(lr, X_test_scaled, y_test, "Logistic Regression")
evaluate(rf, X_test_scaled, y_test, "Random Forest")



===== Logistic Regression =====
ROC-AUC: 0.9725278766314864
PR-AUC: 0.7107483883137459
Confusion Matrix:
 [[69343  1736]
 [   14   109]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     71079
           1       0.06      0.89      0.11       123

    accuracy                           0.98     71202
   macro avg       0.53      0.93      0.55     71202
weighted avg       1.00      0.98      0.99     71202


===== Random Forest =====
ROC-AUC: 0.9779218519826274
PR-AUC: 0.8057270325289216
Confusion Matrix:
 [[70991    88]
 [   23   100]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71079
           1       0.53      0.81      0.64       123

    accuracy                           1.00     71202
   macro avg       0.77      0.91      0.82     71202
weighted avg       1.00      1.00      1.00     71202



real time fraud detection

In [12]:
import time

print("Running real-time simulation...")

stream_n = min(2000, len(X_test_scaled))
stream_X = X_test_scaled[:stream_n]
stream_y = y_test[:stream_n].values

scores, preds = [], []
start = time.time()

for row in stream_X:
    proba = lr.predict_proba(row.reshape(1, -1))[0, 1]
    label = 1 if proba >= 0.5 else 0
    scores.append(proba)
    preds.append(label)

end = time.time()

print("Speed:", stream_n / (end - start), "tx/sec")
print("Streaming ROC-AUC:", roc_auc_score(stream_y, scores))
print("Confusion Matrix:\n", confusion_matrix(stream_y, preds))


Running real-time simulation...
Speed: 3158.97992076762 tx/sec
Streaming ROC-AUC: 0.9996491228070176
Confusion Matrix:
 [[1950   45]
 [   0    5]]


saving models and output

In [14]:
import joblib

os.makedirs("models", exist_ok=True)

joblib.dump(scaler, "models/scaler.pkl")
joblib.dump(lr, "models/logistic_regression.pkl")
joblib.dump(rf, "models/random_forest.pkl")

pd.DataFrame({
    "true_label": stream_y,
    "pred_proba": scores,
    "pred_label": preds,
}).to_csv("stream_output.csv", index=False)

print(" Models & streaming output saved!")


 Models & streaming output saved!
